<a href="https://colab.research.google.com/github/vvvu/potential-chainsaw/blob/main/pytorch-tutorial/%5BIntermediate%5D_Deep_Residual_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper Parameters
num_epochs = 80
batch_size = 100
learning_rate = 0.001

In [3]:
# Image Preprocessing Modules
'''
- transforms.Compose() : Composes several transforms together. 
- transforms.Pad() : 对图片边缘进行填充
- transforms.RandomHorizontalFlip() : 根据概率进行水平翻转
- transforms.RandomCrop() : 随机裁剪
'''
transform = transforms.Compose([
    transforms.Pad(4), 
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()
])

In [4]:
# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True, 
                                             transform=transform,
                                             download=True)

test_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                            train=False, 
                                            transform=transforms.ToTensor())
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)


Extracting ../../data/cifar-10-python.tar.gz to ../../data/


In [5]:
# 3x3 convolution [which means kernel_size = 3]
def conv3x3(in_channels, out_channels, stride = 1):
  return nn.Conv2d(in_channels = in_channels, out_channels = out_channels,
                   kernel_size = 3,
                   stride = stride,
                   padding = 1,
                   bias = False)

In [6]:
# Residual block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True) 
        '''
        nn.ReLU(inplace = False)
        - 此时不会修改输入对象的值，而是返回一个新创建的对象
        nn.ReLU(inplace = True)
        - 修改输入对象的值 
        '''
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

# ResNet
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16) # in_channels = 3, out_channels = 16
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)
        
    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)
    
    def forward(self, x):
      out = self.conv(x)
      out = self.bn(out)
      out = self.relu(out)
      out = self.layer1(out)
      out = self.layer2(out)
      out = self.layer3(out)
      out = self.avg_pool(out)
      out = out.view(out.size(0), -1)
      '''
      - view()的主要作用是将多维的数据，如(none, 36, 2, 2)平铺为一维如(none, 144)。作用
      类似于Keras中的Flatten函数
      '''
      out = self.fc(out)
      return out

In [7]:
model = ResNet(ResidualBlock, layers = [2, 2, 2]).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [8]:
# For updating learning rate
def update_lr(optimizer, lr):
  for param_group in optimizer.param_groups:
    param_group['lr'] = lr

### optimizer.param_groups [Custom Learning Rate]
When you initialize the optimizer using
```python
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
```
or similar optimizer, PyTorch creates one `param_group`. The learning rate is accessible via `param_group['lr']` and the list of parameters is accessible via `param_group['params']`.

**If you want different learning rates for different parameters, you can initialize the optimizer like this:**
```python
optim.Adam([
                {'params': model.base.parameters()},
                {'params': model.classifier.parameters(), 'lr': 1e-3}
            ], lr=1e-2, momentum=0.9)
```
This creates two parameter groups with different learning rates. That is the reason for having `param_groups`

And in this snippet
```python
# For updating learning rate
def update_lr(optimizer, lr):
  for param_group in optimizer.param_groups:
    param_group['lr'] = lr

# Decay learning rate
if (epoch+1) % 20 == 0:
    curr_lr /= 3
    update_lr(optimizer, curr_lr)
```
**We can write our own optimizers in PyTorch. Some tutorials about it we can see on this website: https://mcneela.github.io/machine_learning/2019/09/03/Writing-Your-Own-Optimizers-In-Pytorch.html**

In [9]:
# Train the model
total_step = len(train_loader)
curr_lr = learning_rate
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

    # Decay learning rate
    if (epoch+1) % 20 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)

Epoch [1/80], Step [100/500] Loss: 1.5967
Epoch [1/80], Step [200/500] Loss: 1.5119
Epoch [1/80], Step [300/500] Loss: 1.3726
Epoch [1/80], Step [400/500] Loss: 1.1306
Epoch [1/80], Step [500/500] Loss: 1.0868
Epoch [2/80], Step [100/500] Loss: 1.1260
Epoch [2/80], Step [200/500] Loss: 1.1085
Epoch [2/80], Step [300/500] Loss: 1.0159
Epoch [2/80], Step [400/500] Loss: 0.9186
Epoch [2/80], Step [500/500] Loss: 1.0127
Epoch [3/80], Step [100/500] Loss: 0.7846
Epoch [3/80], Step [200/500] Loss: 1.0951
Epoch [3/80], Step [300/500] Loss: 0.8532
Epoch [3/80], Step [400/500] Loss: 0.8892
Epoch [3/80], Step [500/500] Loss: 0.7544
Epoch [4/80], Step [100/500] Loss: 0.8353
Epoch [4/80], Step [200/500] Loss: 0.9668
Epoch [4/80], Step [300/500] Loss: 0.9323
Epoch [4/80], Step [400/500] Loss: 0.7308
Epoch [4/80], Step [500/500] Loss: 0.8093
Epoch [5/80], Step [100/500] Loss: 0.6724
Epoch [5/80], Step [200/500] Loss: 0.7418
Epoch [5/80], Step [300/500] Loss: 0.6268
Epoch [5/80], Step [400/500] Loss:

In [10]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

Accuracy of the model on the test images: 88.06 %


In [11]:
# Save the model checkpoint
torch.save(model.state_dict(), 'resnet.ckpt')